In [1]:
import os
import pprint
from dotenv import load_dotenv
from typing import List, Optional
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import Qdrant
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langgraph.graph import START, END, StateGraph
from pydantic import BaseModel, Field
from qdrant_client import QdrantClient
from langchain_ollama import OllamaLLM
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import torch
from langchain.embeddings.base import Embeddings

In [2]:
# Cargar variables de entorno 
load_dotenv()
print("Variables de entorno cargadas")

Variables de entorno cargadas


Inicializar modelo LLM

In [3]:
# LLM
llm = OllamaLLM(model="hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:Q8_0")
print("LLM inicializado")
# ollama run hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:Q8_0

LLM inicializado


In [7]:
# Prompt
template = """
Eres un asistente experto en arqueología, historia e información geoespacial.
Tu tarea es responder a la pregunta del usuario basándote **única y exclusivamente** en el siguiente contexto proporcionado.

No utilices ningún conocimiento previo. Si la información no se encuentra en el contexto, responde exactamente: 
"La información solicitada no se encuentra en los documentos proporcionados." Responde en el mismo idioma que la pregunta.

Contexto:
{context}

Pregunta:
{question}

Respuesta concisa y útil:
"""
prompt = ChatPromptTemplate.from_template(template)

__Modelo de embedding all-MiniLM-L6-v2__

In [4]:
# Modelo de Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("Modelo de Embeddings inicializado")

e:\ANACONDA\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
e:\ANACONDA\envs\rag\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Modelo de Embeddings inicializado


In [5]:
# Conexión a Qdrant
client = QdrantClient(url="http://localhost:6333")
vector_store = Qdrant(
    client=client, 
    collection_name="idearq", 
    embeddings=embeddings,
)
print("Conexión a Qdrant establecida con éxito")

Conexión a Qdrant establecida con éxito


C:\Users\elevi\AppData\Local\Temp\ipykernel_24568\2345921672.py:3: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use :class:`~QdrantVectorStore` instead.
  vector_store = Qdrant(


Ingesta de más embeddings

In [6]:
import logging
from pathlib import Path
from langchain.document_loaders import PyMuPDFLoader

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

all_docs = []
folder_path = Path("pdf_articulos_idearq/pdf_articulos_idearq_2subida/") 

logging.info(f"Iniciando la carga de documentos desde: {folder_path.resolve()}")

if not folder_path.is_dir():
    logging.error(f"El directorio especificado no existe: {folder_path.resolve()}")
else:
    for pdf_file in folder_path.glob("*.pdf"):
        try:
            loader = PyMuPDFLoader(str(pdf_file), mode="single")
            docs_from_file = loader.load()
            all_docs.extend(docs_from_file)
            
        except Exception as e:
           
            logging.warning(f"No se pudo cargar el archivo '{pdf_file.name}'. Saltando. Error: {e}")

    logging.info(f"Carga finalizada. Se han cargado un total de {len(all_docs)} documentos.")

2025-08-12 13:31:26,079 - INFO - Iniciando la carga de documentos desde: C:\Users\elevi\Documents\CSIC\RAG\pdf_articulos_idearq\pdf_articulos_idearq_2subida
2025-08-12 13:31:57,072 - INFO - Carga finalizada. Se han cargado un total de 105 documentos.


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

text_splitter_config = {
    "chunk_size": 1000,
    "chunk_overlap": 200, # 15-20% de chunk_size
    "length_function": len,
    "add_start_index": True,
    "separators": ["\n\n", "\n", ". ", " ", ""],
}

text_splitter = RecursiveCharacterTextSplitter(**text_splitter_config)
all_splits = text_splitter.split_documents(all_docs)
logging.info(f"Se dividieron {len(all_docs)} documentos en un total de {len(all_splits)} chunks.")

2025-08-12 13:31:58,804 - INFO - Se dividieron 105 documentos en un total de 27103 chunks.


In [8]:
from langchain_qdrant import Qdrant

QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "idearq"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
client = QdrantClient(url=QDRANT_URL)
logging.info(f"Conectando a la colección '{COLLECTION_NAME}' a través de LangChain...")
vector_store = Qdrant(
    client=client,
    collection_name=COLLECTION_NAME,
    embeddings=embeddings,
)
logging.info("Objeto 'vector_store' listo para la ingesta")
logging.info(f"Iniciando la ingesta de {len(all_splits)} chunks...")
vector_store.add_documents(all_splits)
logging.info("Ingesta completada")

2025-08-12 13:31:58,886 - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-08-12 13:31:58,888 - INFO - Conectando a la colección 'idearq' a través de LangChain...
2025-08-12 13:31:58,891 - INFO - Objeto 'vector_store' listo para la ingesta
2025-08-12 13:31:58,893 - INFO - Iniciando la ingesta de 27103 chunks...
2025-08-12 13:32:04,331 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq/points?wait=true "HTTP/1.1 200 OK"
2025-08-12 13:32:04,788 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq/points?wait=true "HTTP/1.1 200 OK"
2025-08-12 13:32:05,260 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq/points?wait=true "HTTP/1.1 200 OK"
2025-08-12 13:32:05,708 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq/points?wait=true "HTTP/1.1 200 OK"
2025-08-12 13:32:06,308 - INFO - HTTP Request: PUT http://localhost:6333/collections/idearq/points?wait=true "HTTP/1.1 200 OK"
2025-08-12 13:32:06,941 - INFO 

Definir el estado y nodos del grafo

In [100]:
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain_core.documents import Document

# Estado del grafo con Pydantic
class State(BaseModel):
    question: str
    context: List[Document] = Field(default_factory=list)
    answer: Optional[str] = None

# Nodo retrieve
def retrieve(state: State) -> dict:
    retrieved_docs = vector_store.similarity_search(state.question)
    return {"context": retrieved_docs}

# Nodo generate
def generate(state: State) -> dict:
    docs_content = "\n\n".join(doc.page_content for doc in state.context)
    messages = prompt.invoke({"question": state.question, "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response}


In [101]:
from langgraph.graph import START, StateGraph


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [96]:
result = graph.invoke({"question": "¿Cuál es la secuencia cultural de la Prehistoria y Protohistoria de Portugal?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Context: [Document(metadata={'producer': 'iText 4.2.0 by 1T3XT', 'creator': 'Arbortext Advanced Print Publisher 10.0.1465/W Unicode', 'creationdate': '2017-02-09T08:18:37+05:30', 'source': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'file_path': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'total_pages': 12, 'format': 'PDF 1.4', 'title': 'First Evidence of Collective Human Inhumation from the Cardial Neolithic (Cova Bonica, Barcelona, NE Iberian Peninsula)', 'author': 'F. Xavier Oms', 'subject': 'Journal of Field Archaeology, 2016. doi:10.1080/00934690.2016.1260407', 'keywords': '', 'moddate': '2017-02-20T04:16:48-08:00', 'trapped': '', 'modDate': "D:20170220041648-08'00'", 'creationDate': "D:20170209081837+05'30'", 'start_index': 29236, '_id': 'b75bcc2a-f51a-484c-911c-a5b01f4240ef', '_collection_name': 'idearq'}, page_content='Lab. reference\nδ13\nBP\nVar.\nCal BC 2σ\nReferences\nBonica IV2\nCatalonia\nSpain\n6\nHuman bone\nOxA-29639\n19.19\n6351\n35\n5466–5226\nPresent study\nBonica IV2

In [104]:
result = graph.invoke({"question": "¿Cuáles son las dataciones más antiguas para el Neolítico Peninsular?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Context: [Document(metadata={'producer': 'iText 4.2.0 by 1T3XT', 'creator': 'Arbortext Advanced Print Publisher 10.0.1465/W Unicode', 'creationdate': '2017-02-09T08:18:37+05:30', 'source': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'file_path': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'total_pages': 12, 'format': 'PDF 1.4', 'title': 'First Evidence of Collective Human Inhumation from the Cardial Neolithic (Cova Bonica, Barcelona, NE Iberian Peninsula)', 'author': 'F. Xavier Oms', 'subject': 'Journal of Field Archaeology, 2016. doi:10.1080/00934690.2016.1260407', 'keywords': '', 'moddate': '2017-02-20T04:16:48-08:00', 'trapped': '', 'modDate': "D:20170220041648-08'00'", 'creationDate': "D:20170209081837+05'30'", 'start_index': 29236, '_id': 'b75bcc2a-f51a-484c-911c-a5b01f4240ef', '_collection_name': 'idearq'}, page_content='Lab. reference\nδ13\nBP\nVar.\nCal BC 2σ\nReferences\nBonica IV2\nCatalonia\nSpain\n6\nHuman bone\nOxA-29639\n19.19\n6351\n35\n5466–5226\nPresent study\nBonica IV2

In [ ]:
result = graph.invoke({"question": "¿Cuales son las evidencias más antiguas de metalurgia en la Prehistoria de la Península Ibérica?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Context: [Document(metadata={'producer': 'iText 4.2.0 by 1T3XT', 'creator': 'Arbortext Advanced Print Publisher 10.0.1465/W Unicode', 'creationdate': '2017-02-09T08:18:37+05:30', 'source': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'file_path': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'total_pages': 12, 'format': 'PDF 1.4', 'title': 'First Evidence of Collective Human Inhumation from the Cardial Neolithic (Cova Bonica, Barcelona, NE Iberian Peninsula)', 'author': 'F. Xavier Oms', 'subject': 'Journal of Field Archaeology, 2016. doi:10.1080/00934690.2016.1260407', 'keywords': '', 'moddate': '2017-02-20T04:16:48-08:00', 'trapped': '', 'modDate': "D:20170220041648-08'00'", 'creationDate': "D:20170209081837+05'30'", 'start_index': 29236, '_id': 'b75bcc2a-f51a-484c-911c-a5b01f4240ef', '_collection_name': 'idearq'}, page_content='Lab. reference\nδ13\nBP\nVar.\nCal BC 2σ\nReferences\nBonica IV2\nCatalonia\nSpain\n6\nHuman bone\nOxA-29639\n19.19\n6351\n35\n5466–5226\nPresent study\nBonica IV2

In [98]:
result = graph.invoke({"question": "¿Hay evidencias de caballo doméstico en el Calcolítico peninsular?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Context: [Document(metadata={'producer': 'iText 4.2.0 by 1T3XT', 'creator': 'Arbortext Advanced Print Publisher 10.0.1465/W Unicode', 'creationdate': '2017-02-09T08:18:37+05:30', 'source': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'file_path': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'total_pages': 12, 'format': 'PDF 1.4', 'title': 'First Evidence of Collective Human Inhumation from the Cardial Neolithic (Cova Bonica, Barcelona, NE Iberian Peninsula)', 'author': 'F. Xavier Oms', 'subject': 'Journal of Field Archaeology, 2016. doi:10.1080/00934690.2016.1260407', 'keywords': '', 'moddate': '2017-02-20T04:16:48-08:00', 'trapped': '', 'modDate': "D:20170220041648-08'00'", 'creationDate': "D:20170209081837+05'30'", 'start_index': 29236, '_id': 'b75bcc2a-f51a-484c-911c-a5b01f4240ef', '_collection_name': 'idearq'}, page_content='Lab. reference\nδ13\nBP\nVar.\nCal BC 2σ\nReferences\nBonica IV2\nCatalonia\nSpain\n6\nHuman bone\nOxA-29639\n19.19\n6351\n35\n5466–5226\nPresent study\nBonica IV2

In [99]:
result = graph.invoke({"question": "¿Cuándo se produce el poblamiento de Baleares?"})

print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Context: [Document(metadata={'producer': 'iText 4.2.0 by 1T3XT', 'creator': 'Arbortext Advanced Print Publisher 10.0.1465/W Unicode', 'creationdate': '2017-02-09T08:18:37+05:30', 'source': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'file_path': 'pdf_articulos_idearq\\1000_oms_2017.pdf', 'total_pages': 12, 'format': 'PDF 1.4', 'title': 'First Evidence of Collective Human Inhumation from the Cardial Neolithic (Cova Bonica, Barcelona, NE Iberian Peninsula)', 'author': 'F. Xavier Oms', 'subject': 'Journal of Field Archaeology, 2016. doi:10.1080/00934690.2016.1260407', 'keywords': '', 'moddate': '2017-02-20T04:16:48-08:00', 'trapped': '', 'modDate': "D:20170220041648-08'00'", 'creationDate': "D:20170209081837+05'30'", 'start_index': 29236, '_id': 'b75bcc2a-f51a-484c-911c-a5b01f4240ef', '_collection_name': 'idearq'}, page_content='Lab. reference\nδ13\nBP\nVar.\nCal BC 2σ\nReferences\nBonica IV2\nCatalonia\nSpain\n6\nHuman bone\nOxA-29639\n19.19\n6351\n35\n5466–5226\nPresent study\nBonica IV2

In [73]:
point = client.scroll(
    collection_name="idearq",
    limit=1,
    with_vectors=True
)

print(len(point[0][0].vector))  # <- Esta es la dimensión del primer vector insertado


384


In [ ]:
# Benchmark básico para comparar modelos de embeddings en una tarea tipo RAG
from sentence_transformers import SentenceTransformer, util

# Definimos los modelos a evaluar
models = {
    "MiniLM": SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2"),
    "E5-large-instruct": SentenceTransformer("intfloat/multilingual-e5-large-instruct"),
    "E5-large": SentenceTransformer("intfloat/multilingual-e5-large")
}

# Pregunta de ejemplo
query = "¿Dónde se encontraba el poblado argárico de La Bastida?"

# Fragmentos de contexto
docs = [
    "La Bastida fue un asentamiento de la cultura del Argar, ubicado en Totana (Murcia), en el sureste de la península ibérica.",
    "El Imperio romano se extendía desde Britania hasta Egipto, con Roma como su capital.",
    "La cerámica neolítica se caracteriza por sus decoraciones geométricas y su uso en contextos rituales."
]

# Evaluación: similitud coseno entre query y cada documento
results = {}

for name, model in models.items():
    query_emb = model.encode(query, convert_to_tensor=True)
    doc_embs = model.encode(docs, convert_to_tensor=True)
    sims = util.cos_sim(query_emb, doc_embs)[0].tolist()
    results[name] = list(zip(docs, sims))

results


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

{'MiniLM': [('La Bastida fue un asentamiento de la cultura del Argar, ubicado en Totana (Murcia), en el sureste de la península ibérica.',
   0.6460731625556946),
  ('El Imperio romano se extendía desde Britania hasta Egipto, con Roma como su capital.',
   0.3274909257888794),
  ('La cerámica neolítica se caracteriza por sus decoraciones geométricas y su uso en contextos rituales.',
   0.35287004709243774)],
 'E5-large-instruct': [('La Bastida fue un asentamiento de la cultura del Argar, ubicado en Totana (Murcia), en el sureste de la península ibérica.',
   0.9117379784584045),
  ('El Imperio romano se extendía desde Britania hasta Egipto, con Roma como su capital.',
   0.7854597568511963),
  ('La cerámica neolítica se caracteriza por sus decoraciones geométricas y su uso en contextos rituales.',
   0.7597974538803101)],
 'E5-large': [('La Bastida fue un asentamiento de la cultura del Argar, ubicado en Totana (Murcia), en el sureste de la península ibérica.',
   0.8997439742088318),
 

In [18]:
# Benchmark básico para comparar modelos de embeddings en una tarea tipo RAG
from sentence_transformers import SentenceTransformer, util

# Definimos los modelos a evaluar
models = {
    "MiniLM": SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2"),
    "E5-large-instruct": SentenceTransformer("intfloat/multilingual-e5-large-instruct"),
    #"bge-m3": SentenceTransformer("BAAI/bge-m3"),
    "gte": SentenceTransformer("Alibaba-NLP/gte-multilingual-base", model_kwargs={"trust_remote_code": True}),
    "E5-small": SentenceTransformer("intfloat/multilingual-e5-small")
}

# Pregunta de ejemplo
query = "¿Dónde se encontraba el poblado argárico de La Bastida?"

# Fragmentos de contexto
docs = [
    "La Bastida fue un asentamiento de la cultura del Argar, ubicado en Totana (Murcia), en el sureste de la península ibérica.",
    "El Imperio romano se extendía desde Britania hasta Egipto, con Roma como su capital.",
    "La cerámica neolítica se caracteriza por sus decoraciones geométricas y su uso en contextos rituales."
]

# Evaluación: similitud coseno entre query y cada documento
results = {}

for name, model in models.items():
    query_emb = model.encode(query, convert_to_tensor=True)
    doc_embs = model.encode(docs, convert_to_tensor=True)
    sims = util.cos_sim(query_emb, doc_embs)[0].tolist()
    results[name] = list(zip(docs, sims))

results

2025-08-02 16:56:31,358 - INFO - Use pytorch device_name: cpu
2025-08-02 16:56:31,372 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-08-02 16:56:45,039 - INFO - Use pytorch device_name: cpu
2025-08-02 16:56:45,044 - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large-instruct
2025-08-02 16:57:02,529 - INFO - Use pytorch device_name: cpu
2025-08-02 16:57:02,566 - INFO - Load pretrained SentenceTransformer: Alibaba-NLP/gte-multilingual-base


ValueError: Alibaba-NLP/new-impl You can inspect the repository content at https://hf.co/Alibaba-NLP/gte-multilingual-base.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.

In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
import torch
print("PyTorch detecta CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Número de GPUs:", torch.cuda.device_count())
    print("Nombre de la GPU:", torch.cuda.get_device_name(0))
    print("Versión de CUDA:", torch.version.cuda)
else:
    print("No se detecta GPU compatible con CUDA.")

PyTorch detecta CUDA: True
Número de GPUs: 1
Nombre de la GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Versión de CUDA: 12.8


E5-large-instruct devuelve mejor score

In [ ]:
# Benchmark básico para comparar modelos de embeddings en una tarea tipo RAG
import time
import torch
from sentence_transformers import SentenceTransformer, util
import gc # Import the garbage collector module

# --- Comprobar si hay una GPU disponible ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")
print("-" * 30)

# --- SOLUCIÓN: Limpieza de memoria preventiva ---
# A veces, una ejecución anterior puede dejar memoria de la GPU ocupada.
# Esta limpieza inicial asegura que empezamos con la máxima memoria disponible.
print("Realizando limpieza de memoria inicial...")
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Limpieza inicial completada.")
print("-" * 30)


# Definimos los modelos a evaluar
model_names = {
    "MiniLM": "sentence-transformers/all-MiniLM-L6-v2",
    "E5-large-instruct": "intfloat/multilingual-e5-large-instruct",
    "bge-m3": "BAAI/bge-m3",
    "gte": "Alibaba-NLP/gte-multilingual-base",
    "E5-small": "intfloat/multilingual-e5-small",
    "Qwen3": "Qwen/Qwen3-Embedding-0.6B"
}

# Pregunta de ejemplo
query = "¿Dónde se encontraba el poblado argárico de La Bastida?"

# Fragmentos de contexto
docs = [
    "La Bastida fue un asentamiento de la cultura del Argar, ubicado en Totana (Murcia), en el sureste de la península ibérica.",
    "El Imperio romano se extendía desde Britania hasta Egipto, con Roma como su capital.",
    "La cerámica neolítica se caracteriza por sus decoraciones geométricas y su uso en contextos rituales."
]

# --- Bucle único para cargar, evaluar y limpiar ---
results = {}
total_start_time = time.time()

print("\n--- Iniciando evaluación de modelos ---")
for name, model_name in model_names.items():
    print(f"\nProcesando modelo: '{name}' ({model_name})...")
    
    # --- 1. Cargar el modelo ---
    start_time = time.time()
    kwargs = {}
    if name in ["gte", "bge-m3", "Qwen3"]:
        kwargs["trust_remote_code"] = True

    try:
        model = SentenceTransformer(model_name, device=device, **kwargs)
        load_time = time.time() - start_time
        print(f"Modelo '{name}' cargado en {load_time:.2f} segundos.")

        # --- 2. Evaluar el modelo ---
        eval_start_time = time.time()
        query_emb = model.encode(query, convert_to_tensor=True)
        doc_embs = model.encode(docs, convert_to_tensor=True)

        sims = util.cos_sim(query_emb, doc_embs)[0].tolist()
        results[name] = sorted(list(zip(docs, sims)), key=lambda x: x[1], reverse=True)
        eval_time = time.time() - eval_start_time
        print(f"Modelo '{name}' evaluado en {eval_time:.2f} segundos.")

    except Exception as e:
        print(f"Error procesando el modelo {name}: {e}")
        continue
    finally:
        # --- 3. Limpiar la memoria ---
        if 'model' in locals():
            del model
        if 'query_emb' in locals():
            del query_emb
        if 'doc_embs' in locals():
            del doc_embs
        
        gc.collect()
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        print(f"Memoria limpiada para el siguiente modelo.")
        print("-" * 30)

total_end_time = time.time()
print(f"\nBenchmark completado en {total_end_time - total_start_time:.2f} segundos.")

# --- Imprimir resultados finales de forma clara ---
print("\n--- Resultados Finales del Benchmark ---")
for model_name, model_results in results.items():
    print(f"\nResultados para el modelo: {model_name}")
    if model_results:
        doc, score = model_results[0]
        print(f"  Mejor resultado -> Score: {score:.4f} | Doc: {doc}")
    for doc, score in model_results:
        print(f"    Score: {score:.4f} | Doc: {doc}")


## __Evaluación con Ragas__

In [11]:
%pip install ragas

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/767.7 kB ? eta -:--:--
   ------------- -------------------------- 262.1/767.7 kB ? eta -:--:--
   --------------------------- ------------ 524.3/767.7 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 767.7/767.7 kB 1.3 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
    --------------------------------------- 0.5/26.2 MB 1.9 MB/s eta 0:00:14
   - -------------------------------------- 0.8/26.2 MB 1.5 MB/s eta 0:00:17
   - -------------------------------------- 1.0/26.2 MB 1.3 MB/s eta 0:00:20
   - -------------------------------------- 1.0/26.2 MB 1.3 MB/s eta 0:00:20
 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [12]:
import logging
from pathlib import Path
from langchain.document_loaders import PyMuPDFLoader

# Asegúrate de que el logging esté configurado
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Carga robusta de documentos desde un directorio local ---

all_docs = []
folder_path = Path("pdf_articulos_idearq/") # La ruta a tu carpeta de PDFs

logging.info(f"Iniciando la carga de documentos desde: {folder_path.resolve()}")

if not folder_path.is_dir():
    logging.error(f"El directorio especificado no existe: {folder_path.resolve()}")
else:
    # Usamos .glob() que es eficiente para buscar archivos
    for pdf_file in folder_path.glob("*.pdf"):
        try:
            loader = PyMuPDFLoader(str(pdf_file), mode="single")
            docs_from_file = loader.load()
            all_docs.extend(docs_from_file)
            # logging.debug(f"Cargado: {pdf_file.name}") # Descomenta para ver cada archivo
        except Exception as e:
            # Este bloque capturará si un PDF específico está dañado y no se puede leer
            logging.warning(f"No se pudo cargar el archivo '{pdf_file.name}'. Saltando. Error: {e}")

    logging.info(f"Carga finalizada. Se han cargado un total de {len(all_docs)} documentos.")

2025-08-01 15:05:58,482 - INFO - Iniciando la carga de documentos desde: C:\Users\elevi\Documents\CSIC\RAG\pdf_articulos_idearq
2025-08-01 15:07:35,129 - INFO - Carga finalizada. Se han cargado un total de 300 documentos.


In [13]:
# Initialize RAG instance
rag = RAG()

# Load documents
rag.load_documents(all_docs)

# Query and retrieve the most relevant document
query = "¿¿Cuáles son las dataciones más antiguas para el Neolítico Peninsular??"
relevant_doc = rag.get_most_relevant_docs(query)

# Generate an answer
answer = rag.generate_answer(query, relevant_doc)

print(f"Query: {query}")
print(f"Relevant Document: {relevant_doc}")
print(f"Answer: {answer}")

NameError: name 'RAG' is not defined